<a href="https://colab.research.google.com/github/MudassirABBASSi/Paper-pattern-detection/blob/main/paper_Pattern_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install 'git+https://github.com/facebookresearch/detectron2.git'
# !wget -O /content/config.yaml https://raw.githubusercontent.com/facebookresearch/detectron2/main/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml


  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-eyyd9z7a
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-eyyd9z7a
  Resolved https://github.com/facebookresearch/detectron2.git to commit 9604f5995cc628619f0e4fd913453b4d7d61db3f
  Preparing metadata (setup.py) ... done


In [ ]:
# import os, zipfile
# from detectron2.data.datasets import register_coco_instances
# from detectron2.data import MetadataCatalog, DatasetCatalog

# # Paths
# zip_path = "/content/CV Answear sheet.v1i.coco.zip"
# extract_path = "/content/adnan_dataset1"
# train_images = os.path.join(extract_path, "train")
# val_images = os.path.join(extract_path, "valid")
# train_ann = os.path.join(train_images, "_annotations.coco.json")
# val_ann = os.path.join(val_images, "_annotations.coco.json")

# # Extract
# with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#     zip_ref.extractall(extract_path)

# # Register datasets
# register_coco_instances("adnan_train", {}, train_ann, train_images)
# register_coco_instances("adnan_val", {}, val_ann, val_images)


In [16]:
import os
import zipfile

# =================== Step 1: Extract the Dataset ===================
zip_path = "/content/CV Answear sheet.v1i.coco.zip"
extract_path = "/content/CV Answear sheet.v1i.coco"

if not os.path.exists(extract_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall("/content/OMR_Answear_sheet")
    print("Dataset extracted successfully.")
else:
    print("Dataset already extracted.")

# =================== Step 2: Register the Dataset ===================
from detectron2.data.datasets import register_coco_instances
from detectron2.data import DatasetCatalog, MetadataCatalog

# Define dataset paths (update if necessary)
train_images_path = os.path.join(extract_path, "/content/OMR_Answear_sheet/CV Answear sheet.v1i.coco/train")
train_annotations_path = os.path.join(train_images_path, "/content/OMR_Answear_sheet/CV Answear sheet.v1i.coco/train/_annotations.coco.json")
valid_images_path = os.path.join(extract_path, "/content/OMR_Answear_sheet/CV Answear sheet.v1i.coco/valid")
valid_annotations_path = os.path.join(valid_images_path, "/content/OMR_Answear_sheet/CV Answear sheet.v1i.coco/valid/_annotations.coco.json")

# Verify that the paths exist:
assert os.path.exists(train_images_path), f"Train images not found at {train_images_path}"
assert os.path.exists(train_annotations_path), f"Train annotations not found at {train_annotations_path}"
assert os.path.exists(valid_images_path), f"Validation images not found at {valid_images_path}"
assert os.path.exists(valid_annotations_path), f"Validation annotations not found at {valid_annotations_path}"

# Dataset names
train_dataset_name = "cv_train"
valid_dataset_name = "cv_valid"

# Unregister if already registered
if train_dataset_name in DatasetCatalog.list():
    DatasetCatalog.remove(train_dataset_name)
    MetadataCatalog.remove(train_dataset_name)
if valid_dataset_name in DatasetCatalog.list():
    DatasetCatalog.remove(valid_dataset_name)
    MetadataCatalog.remove(valid_dataset_name)

register_coco_instances(train_dataset_name, {}, train_annotations_path, train_images_path)
register_coco_instances(valid_dataset_name, {}, valid_annotations_path, valid_images_path)
print("✅ Training and validation datasets registered successfully.")
print("Registered datasets:", DatasetCatalog.list())

# =================== Step 3: Visualize a Sample (Optional) ===================
import cv2
from detectron2.utils.visualizer import Visualizer

dataset_dicts = DatasetCatalog.get(train_dataset_name)
metadata = MetadataCatalog.get(train_dataset_name)
sample_data = dataset_dicts[0]
sample_image_path = sample_data["file_name"]
sample_image = cv2.imread(sample_image_path)
visualizer = Visualizer(sample_image[:, :, ::-1], metadata=metadata, scale=1.0)
output_vis = visualizer.draw_dataset_dict(sample_data)
cv2.imwrite("sample_image.jpg", output_vis.get_image()[:, :, ::-1])
print("✅ Sample visualization saved as 'sample_image.jpg'.")

# =================== Step 4: Configure the Model ===================
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer, DefaultPredictor

# Enable CUDA blocking for debugging (optional)
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = (train_dataset_name,)
cfg.DATASETS.TEST = (valid_dataset_name,)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
# Increase iterations to 10,000 for better learning.
cfg.SOLVER.MAX_ITER =6000
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 50

# IMPORTANT: Set the number of classes to match your dataset.
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 45

# (Optional) If you don't have a GPU, uncomment the next line:
# cfg.MODEL.DEVICE = "cpu"

print("✅ Model configuration completed.")

# =================== Step 5: Train the Model ===================
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

# =================== Step 6: Inference on a Validation Image ===================
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
predictor = DefaultPredictor(cfg)

valid_images_list = os.listdir(valid_images_path)
if len(valid_images_list) == 0:
    raise ValueError("No validation images found!")
test_image_path = os.path.join(valid_images_path, valid_images_list[0])
test_image = cv2.imread(test_image_path)
outputs = predictor(test_image)

v = Visualizer(test_image[:, :, ::-1], metadata=MetadataCatalog.get(valid_dataset_name), scale=1.0)
result_vis = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2.imwrite("inference_result.jpg", result_vis.get_image()[:, :, ::-1])
print("✅ Inference result saved as 'inference_result.jpg'.")



Dataset extracted successfully.
✅ Training and validation datasets registered successfully.
Registered datasets: ['coco_2014_train', 'coco_2014_val', 'coco_2014_minival', 'coco_2014_valminusminival', 'coco_2017_train', 'coco_2017_val', 'coco_2017_test', 'coco_2017_test-dev', 'coco_2017_val_100', 'keypoints_coco_2014_train', 'keypoints_coco_2014_val', 'keypoints_coco_2014_minival', 'keypoints_coco_2014_valminusminival', 'keypoints_coco_2017_train', 'keypoints_coco_2017_val', 'keypoints_coco_2017_val_100', 'coco_2017_train_panoptic_separated', 'coco_2017_train_panoptic_stuffonly', 'coco_2017_train_panoptic', 'coco_2017_val_panoptic_separated', 'coco_2017_val_panoptic_stuffonly', 'coco_2017_val_panoptic', 'coco_2017_val_100_panoptic_separated', 'coco_2017_val_100_panoptic_stuffonly', 'coco_2017_val_100_panoptic', 'lvis_v1_train', 'lvis_v1_val', 'lvis_v1_test_dev', 'lvis_v1_test_challenge', 'lvis_v0.5_train', 'lvis_v0.5_val', 'lvis_v0.5_val_rand_100', 'lvis_v0.5_test', 'lvis_v0.5_train_coc

roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}


[04/10 10:05:54 d2.engine.train_loop]: Starting training from iteration 0
[04/10 10:05:55 d2.utils.events]:  eta: 0:07:44  iter: 19  total_loss: 9.241  loss_cls: 3.962  loss_box_reg: 0.07356  loss_rpn_cls: 4.66  loss_rpn_loc: 0.4438    time: 0.0777  last_time: 0.0779  data_time: 0.0116  last_data_time: 0.0048   lr: 4.9953e-06  max_mem: 2294M
[04/10 10:05:57 d2.utils.events]:  eta: 0:07:35  iter: 39  total_loss: 6.597  loss_cls: 3.706  loss_box_reg: 0.3658  loss_rpn_cls: 2.125  loss_rpn_loc: 0.4195    time: 0.0772  last_time: 0.0762  data_time: 0.0047  last_data_time: 0.0046   lr: 9.9902e-06  max_mem: 2294M
[04/10 10:05:59 d2.utils.events]:  eta: 0:07:33  iter: 59  total_loss: 4.676  loss_cls: 3.28  loss_box_reg: 0.4019  loss_rpn_cls: 0.5928  loss_rpn_loc: 0.3483    time: 0.0768  last_time: 0.0762  data_time: 0.0044  last_data_time: 0.0047   lr: 1.4985e-05  max_mem: 2294M
[04/10 10:06:00 d2.utils.events]:  eta: 0:07:28  iter: 79  total_loss: 3.65  loss_cls: 2.523  loss_box_reg: 0.4954  

In [13]:
import locale
locale.getpreferredencoding = lambda: 'UTF-8'
!pip install gradio
!pip install torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 120.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [17]:
import torch
import pickle
import numpy as np
import cv2
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

# Set device
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using device: {DEVICE}")

# Load the model from the pickle file
def load_model(model_path):
    try:
        with open(model_path, "rb") as f:
            predictor = pickle.load(f)

        # Ensure model is on the correct device
        predictor.model.to(DEVICE)

        return predictor
    except Exception as e:
        print(f" Error loading model: {e}")
        return None

# Load metadata
def setup_metadata(num_classes=80):
    """Set up dataset metadata for visualization"""
    dataset_name = "paper_detection"

    if dataset_name in MetadataCatalog.list():
        MetadataCatalog.remove(dataset_name)

    MetadataCatalog.get(dataset_name).set(thing_classes=[f"class_{i}" for i in range(num_classes)])

    return MetadataCatalog.get(dataset_name)

# Detect objects in an image
def detect_patterns(input_image):
    if input_image is None:
        return np.zeros((400, 600, 3), dtype=np.uint8), "❌ No image provided. Please upload an image."

    try:
        predictor = load_model("/content/model.pkl")
        if predictor is None:
            return np.zeros((400, 600, 3), dtype=np.uint8), "❌ Model failed to load."

        metadata = setup_metadata()

        # Convert image to OpenCV format
        img_array = np.array(input_image)
        img_array = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)

        # Ensure image is moved to correct device
        img_tensor = torch.tensor(img_array, dtype=torch.float32).permute(2, 0, 1).unsqueeze(0).to(DEVICE)

        # Run inference
        with torch.no_grad():
            outputs = predictor(img_tensor)

        # Visualize results
        v = Visualizer(img_array, metadata=metadata, scale=1.0)
        result = v.draw_instance_predictions(outputs["instances"].to("cpu"))
        result_image = result.get_image()

        return result_image, "✅ Detection completed!"

    except Exception as e:
        return np.zeros((400, 600, 3), dtype=np.uint8), f"❌ Error: {e}"



✅ Using device: cuda


In [25]:
import torch
import torch.nn as nn
import pickle

# Define a simple model (Replace with your actual model)
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc = nn.Linear(10, 2)  # Example: 10 input features → 2 output classes

    def forward(self, x):
        return self.fc(x)

# Initialize model
model = SimpleModel()

# Save the model using pickle
model_path = "model.pkl"
with open(model_path, "wb") as f:
    pickle.dump(model, f)

print(f"✅ Model saved as {model_path}")


✅ Model saved as model.pkl


#image uploade

In [24]:
import gradio as gr
import cv2
import torch
import pandas as pd
import numpy as np
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

# =================== Load the Trained Model ===================
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = "/content/output/model_final.pth"  # Update with your trained model path
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 45  # Ensure this matches your dataset
predictor = DefaultPredictor(cfg)

# Retrieve metadata (Ensure correct dataset is registered)
dataset_name = "cv_valid"  # Update with your actual dataset name
metadata = MetadataCatalog.get(dataset_name)

# =================== Function to Process Image ===================
def detect_objects(image):
    """
    Detect objects in an uploaded image, save annotated image and CSV results.
    """
    im = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  # Convert image from RGB to BGR
    outputs = predictor(im)
    instances = outputs["instances"].to("cpu")

    # =================== Count Detected Classes ===================
    class_counts = {}
    detected_classes = instances.pred_classes.numpy()

    if hasattr(metadata, "thing_classes") and metadata.thing_classes:
        class_names = metadata.thing_classes
    else:
        class_names = [f"Class_{i}" for i in range(cfg.MODEL.ROI_HEADS.NUM_CLASSES)]

    for cls in detected_classes:
        if cls < len(class_names):  # Ensure index is within bounds
            class_name = class_names[cls]
            class_counts[class_name] = class_counts.get(class_name, 0) + 1

    # =================== Format Data for CSV ===================
    formatted_data = []
    for class_name, count in class_counts.items():
        parts = class_name.split("-")  # Split at "-" if present
        if len(parts) > 1:
            formatted_data.append([parts[0], parts[1]])  # Class, Suffix
        else:
            formatted_data.append([class_name, count])  # Class, Count

    # Create formatted DataFrame and Save CSV
    df = pd.DataFrame(formatted_data, columns=["Class", "Count"])
    csv_file = "/content/detection_results.csv"
    df.to_csv(csv_file, index=False)

    # =================== Annotate Image ===================
    visualizer = Visualizer(im[:, :, ::-1], metadata=metadata, scale=1.0)
    out = visualizer.draw_instance_predictions(instances)
    annotated_image = out.get_image()[:, :, ::-1]

    return annotated_image, csv_file  # Return both annotated image and CSV file

# =================== Gradio Interface ===================
iface = gr.Interface(
    fn=detect_objects,
    inputs=gr.Image(type="numpy"),
    outputs=[gr.Image(type="numpy", label="Annotated Image"), gr.File(label="Download CSV Results")],
    title="Object Detection & CSV Export",
    description="Upload an image, detect objects, and download detection results as CSV."
)

iface.launch()


[04/10 10:39:15 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /content/output/model_final.pth ...
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://69c20bf2da0678f0d4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Testing

In [28]:
!pip install easyocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 92.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.8/422.8 kB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 969.6/969.6 kB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.9/292.9 kB 30.0 MB/s eta 0:00:00


In [29]:
import gradio as gr
import cv2
import torch
import pandas as pd
import numpy as np
import easyocr
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

# ========== Load EasyOCR ==========
reader = easyocr.Reader(['en'], gpu=torch.cuda.is_available())

# ========== Load Trained Model ==========
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = "/content/output/model_final.pth"  # Update this path
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 45  # Update to your dataset
predictor = DefaultPredictor(cfg)

# ========== Load Metadata ==========
dataset_name = "cv_valid"  # Replace with your registered dataset name
metadata = MetadataCatalog.get(dataset_name)
class_names = metadata.thing_classes if hasattr(metadata, "thing_classes") else [f"Class_{i}" for i in range(cfg.MODEL.ROI_HEADS.NUM_CLASSES)]

# ========== Format and Order Output Fields ==========
def structure_csv_output(extracted_info):
    ordered_fields = ["Month", "Class", "Section", "Serial"] + [f"B{i}" for i in range(1, 13)]
    result = {key: "" for key in ordered_fields}

    for key, value in extracted_info.items():
        label_clean = key.lower().replace(" ", "")
        if "month" in label_clean:
            result["Month"] = value
        elif "class" in label_clean:
            result["Class"] = value
        elif "section" in label_clean:
            result["Section"] = value
        elif "serial" in label_clean:
            result["Serial"] = value
        elif label_clean.startswith("b") and label_clean[1:].isdigit():
            b_key = f"B{int(label_clean[1:])}"
            if b_key in result:
                result[b_key] = value

    df = pd.DataFrame(list(result.items()), columns=["Field", "Value"])
    csv_path = "/content/student_output.csv"
    df.to_csv(csv_path, index=False)
    return csv_path

# ========== Main Detection Function ==========
def detect_and_extract(image):
    img_bgr = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    outputs = predictor(img_bgr)
    instances = outputs["instances"].to("cpu")
    boxes = instances.pred_boxes.tensor.numpy()
    classes = instances.pred_classes.numpy()

    extracted_info = {}

    for box, cls_id in zip(boxes, classes):
        label = class_names[cls_id]
        x1, y1, x2, y2 = map(int, box)
        cropped = img_bgr[y1:y2, x1:x2]

        # OCR from cropped image
        result = reader.readtext(cropped)
        text = result[0][1] if result else ""

        # Save result
        if label.lower().startswith("serial") or label.lower().startswith("b") or \
           "month" in label.lower() or "class" in label.lower() or "section" in label.lower():
            extracted_info[label] = text

    # Format structured CSV
    csv_file = structure_csv_output(extracted_info)

    # Annotated image
    vis = Visualizer(img_bgr[:, :, ::-1], metadata=metadata, scale=1.0)
    annotated_img = vis.draw_instance_predictions(instances).get_image()[:, :, ::-1]

    return annotated_img, csv_file

# ========== Gradio Interface ==========
iface = gr.Interface(
    fn=detect_and_extract,
    inputs=gr.Image(type="numpy", label="📤 Upload Answer Sheet"),
    outputs=[
        gr.Image(type="numpy", label="📷 Annotated Image"),
        gr.File(label="📥 Download CSV (with actual values)")
    ],
    title="🧠 Answer Sheet Reader with OCR",
    description="Detects fields using AI, extracts actual written values with OCR, and outputs a clean vertical CSV.",
    allow_flagging="never"
)

iface.launch()


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete[04/10 10:52:02 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /content/output/model_final.pth ...


/usr/local/lib/python3.11/dist-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b7dee37e585e5e5dd2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Multiple

In [ ]:
import gradio as gr
import cv2
import torch
import pandas as pd
import numpy as np
import os
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

# =================== Load the Trained Model ===================
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = "/content/output/model_final.pth"  # Update with your trained model path
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 45  # Ensure this matches your dataset
predictor = DefaultPredictor(cfg)

# Retrieve metadata (Ensure correct dataset is registered)
dataset_name = "cv_valid"  # Update with your actual dataset name
metadata = MetadataCatalog.get(dataset_name)

# Define CSV file path
csv_file = "/content/detection_results.csv"

# =================== Function to Process Image ===================
def detect_objects(image):
    """
    Detect objects in an uploaded image, track image number, and save results in a single CSV.
    """
    im = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  # Convert image from RGB to BGR
    outputs = predictor(im)
    instances = outputs["instances"].to("cpu")

    # =================== Count Detected Classes ===================
    class_counts = {}
    detected_classes = instances.pred_classes.numpy()

    if hasattr(metadata, "thing_classes") and metadata.thing_classes:
        class_names = metadata.thing_classes
    else:
        class_names = [f"Class_{i}" for i in range(cfg.MODEL.ROI_HEADS.NUM_CLASSES)]

    for cls in detected_classes:
        if cls < len(class_names):  # Ensure index is within bounds
            class_name = class_names[cls]
            class_counts[class_name] = class_counts.get(class_name, 0) + 1

    # =================== Determine Image Number ===================
    if os.path.exists(csv_file):
        try:
            existing_df = pd.read_csv(csv_file)
            pic_no = existing_df["Pic No"].max() + 1  # Get the last Pic No and increment
        except:
            pic_no = 1  # If CSV is empty or corrupted, start from 1
    else:
        pic_no = 1  # If CSV does not exist, start from 1

    # =================== Format Data for CSV ===================
    formatted_data = []

    for class_name, count in class_counts.items():
        if "-" in class_name:
            # If the class name has a section (e.g., "B6-N"), split it
            main_class, section = class_name.split("-", 1)
            formatted_data.append([pic_no, section, main_class])  # Pic No | Section | Main Class
        else:
            formatted_data.append([pic_no, "Class", count])  # Pic No | Class | Count

    # Convert formatted data to DataFrame
    df = pd.DataFrame(formatted_data, columns=["Pic No", "Section", "Class"])

    # Append data to CSV
    if not os.path.exists(csv_file):
        df.to_csv(csv_file, index=False)  # Create file with headers
    else:
        df.to_csv(csv_file, index=False, header=False, mode="a")  # Append without headers

    # =================== Annotate Image ===================
    visualizer = Visualizer(im[:, :, ::-1], metadata=metadata, scale=1.0)
    out = visualizer.draw_instance_predictions(instances)
    annotated_image = out.get_image()[:, :, ::-1]

    return annotated_image, csv_file  # Return both annotated image and CSV file

# =================== Gradio Interface ===================
iface = gr.Interface(
    fn=detect_objects,
    inputs=gr.Image(type="numpy"),
    outputs=[gr.Image(type="numpy", label="Annotated Image"), gr.File(label="Download CSV Results")],
    title="Object Detection & CSV Export",
    description="Upload multiple images, detect objects, and save results into a single CSV file with image numbers."
)

iface.launch()


[03/24 17:55:10 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /content/output/model_final.pth ...
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5d65d50108d6f2871f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
